# Calculate BIC curves

This notebook will fit 2 to 20 class GMM models to three UK-ESM historical ensemble members, caculate the BIC score for each, and save to model_19651995/BICs2-30.obj. This is required to reproduce Figure 2 from *Heuristic Methods for Determining the Number of Classes in Unsupervised Classification of Climate Models*, E. Boland et al. 2022 (doi to follow). This requires cluster_utils.py and input datafiles via the googleapi CMIP6 store (see cluster_utils.py for more info)

Please attribute any plots or code from this notebook using the DOI from Zenodo: to come

Updated Nov 2022
E Atkinson & E Boland [emmomp@bas.ac.uk](email:emmomp@bas.ac.uk)

In [70]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:39925")
client

<Client: 'tcp://127.0.0.1:39925' processes=1 threads=1, memory=8.00 GiB>

In [67]:
client.close()

2022-11-30 12:47:19,177 - distributed.client - ERROR - 
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/comm/core.py", line 291, in connect
    comm = await asyncio.wait_for(
  File "/srv/conda/envs/notebook/lib/python3.9/asyncio/tasks.py", line 479, in wait_for
    return fut.result()
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/comm/tcp.py", line 461, in connect
    convert_stream_closed_error(self, e)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/comm/tcp.py", line 142, in convert_stream_closed_error
    raise CommClosedError(f"in {obj}: {exc.__class__.__name__}: {exc}") from exc
distributed.comm.core.CommClosedError: in <distributed.comm.tcp.TCPConnector object at 0x7f3b733c3640>: ConnectionRefusedError: [Errno 111] Connection refused

During ha

In [1]:
from dask_gateway import Gateway
gateway = Gateway("https://dask-gateway.jasmin.ac.uk", auth="jupyterhub")
from dask.distributed import Client

'''properly shutdown any previous clusters'''
clusters=gateway.list_clusters()
if clusters != []:
    print(f'found {len(clusters)} clusters')
    for cluster in clusters:
        cluster = gateway.connect(cluster.name)
        client=Client(cluster)
        client.close()
        cluster.shutdown()

found 1 clusters


/home/users/eboland/.conda/envs/nbenvconda/lib/python3.9/site-packages/distributed/client.py:1148: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+---------+
| Package     | client    | scheduler | workers |
+-------------+-----------+-----------+---------+
| cloudpickle | 1.6.0     | 2.1.0     | None    |
| dask        | 2021.05.0 | 2022.7.0  | None    |
| distributed | 2021.05.0 | 2022.7.0  | None    |
| lz4         | None      | 4.0.0     | None    |
| msgpack     | 1.0.2     | 1.0.4     | None    |
| toolz       | 0.11.1    | 0.12.0    | None    |
+-------------+-----------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [2]:
import dask_gateway

# Create a connection to dask-gateway.
gw = dask_gateway.Gateway("https://dask-gateway.jasmin.ac.uk", auth="jupyterhub")

# Inspect and change the options if required before creating your cluster.
options = gw.cluster_options()
options.worker_cores = 4

# Create a dask cluster, or, if one already exists, connect to it.
# This stage creates the scheduler job in SLURM, so may take some time.
# While your job queues.
clusters = gw.list_clusters()
if not clusters:
    cluster = gw.new_cluster(options, shutdown_on_close=False)
else:
    cluster = gw.connect(clusters[0].name)

# Create at least one worker, and allow your cluster to scale to three.
cluster.adapt(minimum=1, maximum=3)

# Get a dask client.
client = cluster.get_client()

In [3]:
import numpy as np
import os
import pickle
import cluster_utils as flt
from sklearn import metrics

### User options
Leave as is to recreate the paper

In [4]:
# Number of classes 
model_folder='model_20012017'
max_classes = 20 #max classes
#Time range
tslice=slice('2001-01', '2017-12') 
#Depth range
levSel=slice(5, 2000)
ids = ['r1i1p1f2', 'r2i1p1f2', 'r3i1p1f2']
ntrain=3000 #number of profiles per month to use in training dataset
npca=3

Uncomment the following three lines if you need to generate mask.npy:

In [5]:
#data = flt.retrieve_profiles(timeRange = slice('1995-01', '1995-02'),levSel=levSel)
#np.save('data/mask_3000', data['n'])
#mask=data['n']
mask = np.load('data/mask_3000.npy', allow_pickle=True)

### Fit 2-30 class models for each ensemble member
Saves each individual PCA model, GMM model and BIC/AIC score to \[model_folder\]

Saves all BICs/AICs to \[model_folder\]/\[BICs/AICs\]2-30.obj

In [ ]:
BICs = {}
AICs = {}
SILs = {}
for m_id in ids:
    path_id = '{}/{}'.format(model_folder, m_id)
    if not os.path.isdir(path_id):
        os.makedirs(path_id)
    print('Starting {}'.format(m_id))
    options = {'memberId' : m_id}
    
    # Load training set
    [data,pca] = flt.generate_trainingset(timeRange = tslice, mask=mask, options=options,N=ntrain,n_components=npca,levSel=levSel)
    
    bic = np.zeros(max_classes-1)
    aic = bic.copy()
    sil = bic.copy()
    
    with open('{}/pca.obj'.format(path_id), 'wb') as file:
        pickle.dump(pca, file)
        
    print('Finished setup for {}'.format(m_id))
    
    for n_classes in range(2, max_classes+1):
        
        path_n = '{}/{}/{}'.format(model_folder, m_id, n_classes)
        
        if not os.path.isdir(path_n):
            os.makedirs(path_n)
            
        if os.path.exists('{}/gmm.obj'.format(path_n)):
            with open('{}/gmm.obj'.format(path_n), 'rb') as file:
                gmm=pickle.load(file)
        else:
            gmm = flt.train_gmm(data, n_classes)
            with open('{}/gmm.obj'.format(path_n), 'wb') as file:
                pickle.dump(gmm, file)
        
        if os.path.exists('{}/bic.obj'.format(path_n)):
            with open('{}/bic.obj'.format(path_n), 'rb') as file:
                bic[n_classes-2] = pickle.load(file)
        else:
            bic[n_classes-2] = gmm.bic(data)
            with open('{}/bic.obj'.format(path_n), 'wb') as file:
                pickle.dump(bic[n_classes-2],file)
        
        if os.path.exists('{}/aic.obj'.format(path_n)):
            with open('{}/aic.obj'.format(path_n), 'rb') as file:
                aic[n_classes-2] = pickle.load(file)
        else:        
            aic[n_classes-2] = gmm.aic(data)
            with open('{}/aic.obj'.format(path_n), 'wb') as file:
                pickle.dump(aic[n_classes-2],file)     

     #   if os.path.exists('{}/sil.obj'.format(path_n)):
     #       with open('{}/sil.obj'.format(path_n), 'rb') as file:
     #           sil[n_classes-2] = pickle.load(file)
     #   else:       # Calculate silhouette score for 10000 point sample        
        inds=np.random.randint(0,data.shape[0],10000)
        labels=flt.gmm_classify(data[inds,:],gmm)
        sil[n_classes-2]=metrics.silhouette_score(data[inds,:],labels,n_jobs=-1)
        sample_silhouette_values = metrics.silhouette_samples(data[inds,:],labels,n_jobs=-1)
        with open('{}/sil.obj'.format(path_n), 'wb') as file:
            pickle.dump(sil[n_classes-2],file)
        with open('{}/sil_vals.obj'.format(path_n), 'wb') as file:
            pickle.dump(sample_silhouette_values,file)
        with open('{}/sil_labels.obj'.format(path_n), 'wb') as file:
            pickle.dump(labels,file)
                
        
        print('Finished {} with {} classes'.format(m_id, n_classes))
        
    BICs[m_id] = bic
    AICs[m_id] = aic
    SILs[m_id] = sil
    
with open('{}/BICs2-20.obj'.format(model_folder), 'wb') as file:
    pickle.dump(BICs, file)
with open('{}/AICs2-20.obj'.format(model_folder), 'wb') as file:
    pickle.dump(AICs, file)
with open('{}/SILs2-20.obj'.format(model_folder), 'wb') as file:
    pickle.dump(SILs, file)

print('Done!')

Starting r1i1p1f2
Finished setup for r1i1p1f2


/home/users/eboland/.conda/envs/nbenvconda/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 1.1.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Finished r1i1p1f2 with 2 classes


/home/users/eboland/.conda/envs/nbenvconda/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 1.1.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Finished r1i1p1f2 with 3 classes


...

/home/users/eboland/.conda/envs/nbenvconda/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 1.1.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Finished r1i1p1f2 with 6 classes


/home/users/eboland/.conda/envs/nbenvconda/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 1.1.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Finished r1i1p1f2 with 7 classes


/home/users/eboland/.conda/envs/nbenvconda/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 1.1.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [8]:
import importlib
importlib.reload(flt)

<module 'cluster_utils' from '/home/users/eboland/bas-pangeo-backup/emma-volume-backup2/emma-volume-backup/OceanClustering_paper/cluster_utils.py'>

In [39]:
data=flt.retrieve_profiles_JASMIN(timeRange = tslice, mask=mask, options=options,levSel=levSel)

#[data,pca] = flt.generate_trainingset(timeRange = tslice, mask=mask, options=options,N=ntrain,n_components=npca,levSel=levSel)

OSError: no files to open

In [40]:
import sys
sys.path.insert(0,'/home/users/eboland/python')
import baspy as bp
import xarray as xr

In [41]:
    options_default = {
      'dataVariableId' : 'thetao',
      'dataSourceId' : 'UKESM1-0-LL',
      'experimentId' : 'historical',
      'tableId' : 'Omon',
      'memberId' : 'r1i1p1f2',
      'raw' : False
  }
    options = {**options_default, **options}
    
    df = bp.catalogue(dataset='cmip6',Var=options['dataVariableId'],
                      CMOR=options['tableId'],Model=options['dataSourceId'],
                      Experiment=options['experimentId'],RunID=options['memberId'])

In [42]:
 filesRaw = bp.open_dataset(df)

CancelledError: ('open_dataset-dad37783642a8efd51d08aad7ac9f863latitude-96ec6ad197ff5c3343b98dcc5a8606f2', 0, 0)

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
